# Requirements

In [ ]:
import os, sys

pkg_path = 'D:\\Projects\\System_Trading\\JnQ\\'

os.chdir(pkg_path)

# mpl_finance_path = 'D:\\python\\python38_1\\projects\\JnQ\\mpl_finance'
# ta_lib_path = 'D:\\python\\python38_1\\projects\\JnQ\\ta_lib'
funcs_path = pkg_path + 'funcs'

if funcs_path not in sys.path:

  try:
    # sys.path.insert(0, '/content/drive/My Drive/Colab Notebooks/JnQ')
    sys.path.insert(0, pkg_path + 'Bank')
    sys.path.insert(0, funcs_path)
    # sys.path.insert(0, mpl_finance_path)
    # sys.path.insert(0, ta_lib_path)
    
  except Exception as e:
    print(e)

In [ ]:
import os
import talib
from funcs.public.idep import *
from funcs.public.plot_check import *
from funcs.public.en_ex_pairing import *
from funcs.public.indicator import *
from funcs.public.broker import *
from funcs.public.ds import *
from ast import literal_eval
import logging
import importlib


import matplotlib.pyplot as plt
from matplotlib import gridspec

# import torch

import numpy as np
import pandas as pd
import scipy.stats as stats
# from sklearn.metrics.pairwise import cosine_similarity

# import bz2
import pickle
# import _pickle as cPickle
import shutil
import json
from easydict import EasyDict
import copy

import datetime
from datetime import datetime
import random
import time
# import warnings

from IPython.display import clear_output
# warnings.simplefilter("ignore", category=RuntimeWarning)

np.seterr(invalid="ignore")
np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=2000) 

pd.set_option('mode.chained_assignment',  None)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Telegramer

In [ ]:
import telegram
from telegram.ext import Updater
from telegram.ext import MessageHandler, Filters

In [ ]:
def echo(self, update, context):
    self.user_text = update.message.text

In [ ]:
#       i. Telegram logger
#               1. chat_id 는 env 동일.
token = "6717940201:AAFc62YReeED3mJKw5drHnT7jp6dzXy0rPE"
msg_bot = telegram.Bot(token=token)
chat_id = "5320962614"

#       ii. Telegram messenger
#           1. init.
user_text = None

updater = Updater(token=token, use_context=True)
dispatcher = updater.dispatcher

echo_handler = MessageHandler(Filters.text & (~Filters.command), echo)
dispatcher.add_handler(echo_handler)

#           2. polling for messenger.
updater.start_polling()

In [ ]:
msg = "ping telegramer"

try:
    msg_bot.sendMessage(chat_id=chat_id, text=msg)
except Exception as e:
    print("error in msg_bot : {}".format(e))

# Get data

## Upbit

In [ ]:
fluc_arr = np.array(list(fluc_dict.items()))

fluc_arr_krw = np.array([v_ for v_ in fluc_arr if 'krw'.upper() in v_[0]])
fluc_arr_krw = fluc_arr_krw[fluc_arr_krw[:, 1].astype(float).argsort()[::-1]]

In [ ]:
pd.DataFrame(fluc_arr_krw).set_index(0).to_excel("signal/upbit_fluc_max.xlsx")

In [ ]:
from funcs.upbit.concat_candlestick_ftr_v2 import concat_candlestick
from pyupbit import get_tickers

"""
1. upbit 에서 당일 기준 이전 데이터를 조회할지라도 None 은 성립하지 않는다. --> end_date = today 로 변경 요구.
"""


days = 1  # 330 3

end_date = None  # None 2023-07-04 # "2023-01-06" "2021-04-12" "2021-03-23"
    
intervals = [ 'D']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old
intervals = ['30m', '1h', '2h']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old
intervals = ['1d']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old

time_schedule = {'4h': [21, 17, 13, 9, 5, 1], '1d': [9]}
    
show_process = False
show_process = True



with open("signal/upbit_fluc_max.pkl", 'rb') as f:
    fluc_dict = pickle.load(f)
    
# hour_prev = None    
symbol_list = get_tickers()

while 1:

    time.sleep(1)  # term for small fan sound.
    
    target_symbol_total = []
    for interval in intervals:
        
        timestamp = datetime.now().timestamp()
        hour_current = datetime.fromtimestamp(datetime.now().timestamp()).hour
        hour_prev = datetime.fromtimestamp(datetime.now().timestamp() - 10).hour  # 10 seconds before.
        
        # print("hour_current :", hour_current)
        # print("hour_prev :", hour_prev)
        # print("time_schedule[interval] :", time_schedule[interval])
        
        if not ((hour_current in time_schedule[interval]) and hour_current != hour_prev):
            continue

        target_symbol = []
        for s_i, symbol in enumerate(symbol_list):

            print(s_i)

            if 'KRW' not in symbol:
                continue

            try:
                concated_df, end_date = concat_candlestick(symbol, 
                                                           interval, 
                                                           days, 
                                                           limit=limit_by_itv(interval),
                                                           end_date=end_date, 
                                                           show_process=show_process, 
                                                           timesleep=0.1)


                concated_df2 = fisher_v2(concated_df, 30, itv=interval)    
                # display(concated_df2.tail())
                # break

                target_data = concated_df2.iloc[-2:, -1].to_numpy()

                """
                condition
                """
                # 1. fisher band
                fisher_lower = -1.5
                if target_data[0] < fisher_lower < target_data[1]:
                    target_symbol.append(symbol)

                clear_output(wait=True)          



                # break
                # save_path = os.path.join(save_dir, save_name)
                # concated_df.reset_index().to_feather(save_path, compression='lz4')
                # print(save_path, "saved.\n")
            except Exception as e:
                print("error in save to_feather :", e)
                continue

        target_symbol_total = np.array([[s_.replace('KRW-', ''), fluc_dict[s_]] for s_ in target_symbol])
        if len(target_symbol_total) > 0:
            target_symbol_total = target_symbol_total[target_symbol_total[:, 1].astype(float).argsort()[::-1]]
            print("target_symbol_total :", target_symbol_total)

            msg = "target_symbol_total : {} {}".format(interval, target_symbol_total)
            try:
                msg_bot.sendMessage(chat_id=chat_id, text=msg)
            except Exception as e:
                print("error in msg_bot : {}".format(e))
        # break

In [ ]:
df_res = pd.DataFrame(index=intervals, data=target_symbol_total)
df_res

In [ ]:
df_res.to_excel("signal/upbit_res.xlsx")

### get max price in whole range

In [ ]:

days = 365 * 10  # 330 3
end_date = str(datetime.now()).split(' ')[0]    
count = int((days * 60 * 24) / itv_to_number(interval))
timesleep = 0.1
print("count :", count)


fluc_dict = {}
for s_i, symbol in enumerate(get_tickers()):
    
    print(s_i, symbol)
    
    df = pyupbit.get_ohlcv("{}".format(symbol),
                                       interval=itv_binance_to_upbit(interval),
                                       count=count,
                                       period=timesleep,
                                       to=end_date) 

    print(df.index[0], end=" --> ")
    print(df.index[-1])

    fluc_max = df.high.max() / df.close.iloc[-1]
    fluc_dict[symbol] = fluc_max
    
    clear_output(wait=True)
    

In [ ]:
with open("signal/upbit_fluc_max.pkl", 'wb') as f:
    pickle.dump(fluc_dict, f)

## Binance_Futures

In [ ]:
# with open(r"D:\Projects\System_Trading\JnQ\Bank\tickers\binance_20240102.pkl", 'wb') as f:
#     pickle.dump(symbol_list, f)

In [ ]:
from binance.um_futures import UMFutures
from funcs.binance.futures_concat_candlestick_ftr_v2 import *
# break


days = 5  # 330 3

end_date = None  # None 2023-07-04 # "2023-01-06" "2021-04-12" "2021-03-23"
    
intervals = ['4h', '1d']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old
intervals = ['15m', '30m', '1h', '4h', '1d']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old

limit = 1500
    
show_process = False
show_process = True


 
symbol_list = [s_['symbol'] for s_ in um_futures_client.exchange_info()['symbols']]



target_symbol_total = []
for interval in intervals:
    
    target_symbol = []
    for s_i, symbol in enumerate(symbol_list):
        
        print(s_i)
        
        # if 'KRW' not in symbol:
        #     continue
        
        try:
            concated_df, end_date = concat_candlestick_v2(symbol, 
                                                          interval, 
                                                          days,
                                                          limit=limit,
                                                          end_date=end_date,
                                                          show_process=True,
                                                          timesleep=0.4)
            
            
            concated_df2 = fisher_v2(concated_df, 30, itv=interval.upper())      
            # display(concated_df2.tail())
            # break
            
            target_data = concated_df2.iloc[-2:, -1].to_numpy()
            
            """
            condition
            """
            # 1. fisher band
            fisher_lower = -1.5
            if target_data[0] < fisher_lower < target_data[1]:
                target_symbol.append(symbol)
                
            clear_output(wait=True)          
            
            
            
            # break
            # save_path = os.path.join(save_dir, save_name)
            # concated_df.reset_index().to_feather(save_path, compression='lz4')
            # print(save_path, "saved.\n")
        except Exception as e:
            print("error in save to_feather :", e)
            continue
     
    target_symbol_total.append([s_ + '.P' for s_ in target_symbol])
    # print("target_symbol_total :", target_symbol_total)
    # break

In [ ]:
df_res = pd.DataFrame(index=intervals, data=target_symbol_total)
df_res

In [ ]:
df_res.to_excel("signal/binance_futures_res.xlsx")